# House Price Model Pipeline - Scoring New Data

Here is a potential solution approach to Model Scoring

In [1]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# for the yeo-johnson transformation
import scipy.stats as stats

# to save the model
import joblib

In [2]:
# load the unseen / new dataset
data = pd.read_csv('test.csv')

# rows and columns of the data
print(data.shape)

# visualise the dataset
data.tail()

(5911, 16)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,age_group,income_bracket,rooms_per_household,bedrooms_per_household,population_per_household,median_income_per_household
5906,-121.90,37.36,47.0,1007.0,245.0,581.0,240.0,2.9545,237500.0,<1H OCEAN,middle-aged,low-income,4.0,1.0,2.0,0.012310
5907,-117.32,34.11,41.0,1229.0,302.0,994.0,270.0,1.4891,67300.0,INLAND,middle-aged,low-income,5.0,1.0,4.0,0.005515
5908,-117.91,33.67,16.0,7961.0,2276.0,5014.0,2116.0,3.5120,218400.0,<1H OCEAN,minor,low-income,4.0,1.0,2.0,0.001660
5909,-121.28,37.82,10.0,9205.0,1774.0,5935.0,1673.0,3.6500,119400.0,INLAND,minor,low-income,6.0,1.0,4.0,0.002182
5910,-118.24,34.15,17.0,5282.0,1605.0,4116.0,1574.0,3.0520,209800.0,<1H OCEAN,minor,low-income,3.0,1.0,3.0,0.001939


In [3]:
# drop the target and varibles removed from the train sat
data.drop(['total_bedrooms', 'households', 'age_group', 'median_house_value'], axis=1, inplace=True)

data.shape

(5911, 12)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5911 entries, 0 to 5910
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   longitude                    5911 non-null   float64
 1   latitude                     5911 non-null   float64
 2   housing_median_age           5911 non-null   float64
 3   total_rooms                  5911 non-null   float64
 4   population                   5911 non-null   float64
 5   median_income                5911 non-null   float64
 6   ocean_proximity              5911 non-null   object 
 7   income_bracket               5911 non-null   object 
 8   rooms_per_household          5911 non-null   float64
 9   bedrooms_per_household       5711 non-null   float64
 10  population_per_household     5911 non-null   float64
 11  median_income_per_household  5911 non-null   float64
dtypes: float64(10), object(2)
memory usage: 554.3+ KB


## Feature Engineering

We need to transform the data based on what we did to the train dataset during the Feature Engineering Phase:

    Missing Values
    Temporal Values
    Non-Gaussian distributed values
    Categorical features: remove rare labels, if any
    Categorical Features: convert strings to numbers
    Put the variables in a similar scale
    
### Missing Values

#### Categorical Features

Replacing missing values with the string "missing" in those variables with missing data. Alternatively, we will replace missing data with the most frequent category in those variables which contain fewer observations without values.

In [5]:
# let's identify the categorical variables
# we will capture those of type object
# remove age_group, because it is not in the training dataset
cat_vars = [var for var in data.columns if data[var].dtype == 'O']

# number of categorical variables
len(cat_vars)

2

In [6]:
# make a list of the categorical variables that contain missing values

cat_vars_with_na = [
    var for var in cat_vars
    if data[var].isnull().sum() > 0
]

# print percentage of missing values per variable
data[cat_vars_with_na ].isnull().mean().sort_values(ascending=False)

Series([], dtype: float64)

In [7]:
# variables to impute with the string missing
with_string_missing = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean() > 0.1]

# variables to impute with the most frequent category
with_frequent_category = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean() < 0.1]

with_string_missing

[]

In [8]:
with_frequent_category

[]

In [9]:
# replace missing values with new label: "Missing"

data[with_string_missing] = data[with_string_missing].fillna('Missing')

In [10]:
for var in with_frequent_category:
    
    # there can be more than 1 mode in a variable
    # we take the first one with [0]    
    mode = data[var].mode()[0]
    
    print(var, mode)
    
    data[var].fillna(mode, inplace=True)

## Numerical Features

To engineer missing values in numerical variables, we will:

- add a binary missing indicator variable
- and then replace the missing variables in the original variable with the mean

In [11]:
# now let's identify the numerical variables

num_vars = [
    var for var in data.columns if var not in cat_vars
]

# number of numerical variables
len(num_vars)

10

In [12]:
# make a list with the numerical variables that contain missing values
vars_with_na = [
    var for var in num_vars
    if data[var].isnull().sum() > 0
]

# print percentage of missing values per variable
data[vars_with_na].isnull().mean()

bedrooms_per_household    0.033835
dtype: float64

In [13]:
# replace missing values as we described above

for var in vars_with_na:

    # calculate the mean using the train set
    mean_val = data[var].mean()
    
    print(var, mean_val)

    # add binary missing indicator (in train and test)
    data[var + '_na'] = np.where(data[var].isnull(), 1, 0)

    # replace missing values by the mean
    # (in train and test)
    data[var].fillna(mean_val, inplace=True)
    
# check that we have no more missing values in the engineered variables
data[vars_with_na].isnull().sum()

bedrooms_per_household 1.0427245666258098


bedrooms_per_household    0
dtype: int64

In [14]:
data[['bedrooms_per_household_na']].head()

,bedrooms_per_household_na
0,1
1,1
2,1
3,1
4,1


## Numerical variable transformation

#### Yeo-Johnson Transformation

We will apply the Yeo-Johnson Transformation to the continous variables, except `housing_median_age`

In [15]:
# Capture Discrete and Continous features
discrete_vars = [var for var in num_vars if len(data[var].unique()) < 20]

cont_vars = [var for var in num_vars if var not in discrete_vars+["longitude", "latitude"]]

# Capture the Skewed variables from the continous variables for later

skewed = ["population_per_household", "median_income_per_household"]

# Capture the remaining continuous features
# remove the housing_median_age which already has good spread
cont_vars = [item for item in cont_vars if item not in skewed and item != 'housing_median_age']
cont_vars

['total_rooms', 'population', 'median_income', 'rooms_per_household']

In [16]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,population,median_income,ocean_proximity,income_bracket,rooms_per_household,bedrooms_per_household,population_per_household,median_income_per_household,bedrooms_per_household_na
0,-119.01,36.06,25.0,1505.0,1392.0,1.6812,INLAND,low-income,4.0,1.042725,4.0,0.004683,1
1,-119.46,35.14,30.0,2943.0,1565.0,2.5313,INLAND,low-income,5.0,1.042725,3.0,0.004334,1
2,-118.72,34.28,17.0,3051.0,1705.0,5.7376,<1H OCEAN,middle-income,6.0,1.042725,3.0,0.011591,1
3,-121.93,36.62,34.0,2351.0,1063.0,3.7250,NEAR OCEAN,low-income,5.0,1.042725,2.0,0.008703,1
4,-117.61,34.08,12.0,4427.0,2400.0,4.7147,INLAND,low-income,5.0,1.042725,3.0,0.005593,1
5,-118.02,33.89,36.0,1375.0,670.0,5.0839,<1H OCEAN,middle-income,6.0,1.042725,3.0,0.023004,1
6,-118.08,33.92,38.0,1335.0,1011.0,3.6908,<1H OCEAN,low-income,5.0,1.042725,4.0,0.013720,1
7,-122.08,37.39,4.0,2292.0,1050.0,4.8036,NEAR BAY,low-income,4.0,1.042725,2.0,0.008225,1
8,-118.23,34.18,45.0,2332.0,943.0,8.1132,<1H OCEAN,middle-income,7.0,1.042725,3.0,0.023933,1
9,-120.37,38.01,30.0,473.0,242.0,2.5417,INLAND,low-income,5.0,1.042725,3.0,0.027330,1


In [17]:
# note how we use the lambda that we learned from the train set
# in the notebook on Feature Engineering.

cont_var_map = {
    'total_rooms': 0.21950357587206626,
    'population': 0.2320775156661516,
    'median_income': 0.03910515153028596,
    'rooms_per_household': -0.48608895100991023,
}

# Note that we need to hard code this value

# data['total_rooms'] = stats.yeojohnson(data['total_rooms'], lmbda=-12.55283001172003)

for var, lmbda in cont_var_map.items():
    data[var] = stats.yeojohnson(data[var], lmbda=lmbda)

In [18]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,population,median_income,ocean_proximity,income_bracket,rooms_per_household,bedrooms_per_household,population_per_household,median_income_per_household,bedrooms_per_household_na
0,-119.01,36.06,25.0,18.148264,18.812131,1.005530,INLAND,low-income,1.116382,1.042725,4.0,0.004683,1
1,-119.46,35.14,30.0,21.747106,19.448897,1.293308,INLAND,low-income,1.196176,1.042725,3.0,0.004334,1
2,-118.72,34.28,17.0,21.955940,19.925736,1.980665,<1H OCEAN,middle-income,1.258338,1.042725,3.0,0.011591,1
3,-121.93,36.62,34.0,20.482341,17.410705,1.600986,NEAR OCEAN,low-income,1.196176,1.042725,2.0,0.008703,1
4,-117.61,34.08,12.0,24.212553,21.926027,1.803819,INLAND,low-income,1.196176,1.042725,3.0,0.005593,1
5,-118.02,33.89,36.0,17.702790,15.206864,1.870922,<1H OCEAN,middle-income,1.258338,1.042725,3.0,0.023004,1
6,-118.08,33.92,38.0,17.559121,17.159598,1.593268,<1H OCEAN,low-income,1.196176,1.042725,4.0,0.013720,1
7,-122.08,37.39,4.0,20.343105,17.348827,1.820350,NEAR BAY,low-income,1.116382,1.042725,2.0,0.008225,1
8,-118.23,34.18,45.0,20.437803,16.815819,2.308007,<1H OCEAN,middle-income,1.308546,1.042725,3.0,0.023933,1
9,-120.37,38.01,30.0,13.060023,11.108544,1.296398,INLAND,low-income,1.196176,1.042725,3.0,0.027330,1


### Binarize Skewed Variables

In [19]:
skewed

['population_per_household', 'median_income_per_household']

In [20]:
for var in skewed:
    data[var] = np.where(data[var]<=1, 0, 1)

### Categorical Features: Apply Mappings

This is applied on the `income_bracket` feature

In [21]:
# re-map strings to numbers, which determine quality

income_mappings = {'low-income': 1, 'middle-income': 2, 'high-income': 3}

data['income_bracket'] = data['income_bracket'].map(income_mappings)

In [22]:
# check absence of na in the data set

with_null = [var for var in data.columns if data[var].isnull().sum() > 0]

with_null

[]

Great! No more missing values!!

## Removing Rare Labels

We will group these categories that are present in less than 1% of the observations, and present them in the category called `Rare`

In [23]:
# Capturing the remaining Categorical Features

cat_others = [var for var in cat_vars if var != 'income_bracket']

cat_others

['ocean_proximity']

In [24]:
def find_frequent_labels(df, var, rare_perc):
    # function finds the labels that are shared by more than
    # a certain % of the houses in the dataset
    df = df.copy()
    tmp = df.groupby(var)[var].count() / len(df)
    return tmp[tmp > rare_perc].index


for var in cat_others:
    # find the frequent categories
    frequent_ls = find_frequent_labels(data, var, 0.01)
    print(var, frequent_ls)
    print()
    # replace rare categories by the string "Rare"
    data[var] = np.where(data[var].isin(
        frequent_ls), data[var], 'Rare')

ocean_proximity Index(['<1H OCEAN', 'INLAND', 'NEAR BAY', 'NEAR OCEAN'], dtype='object', name='ocean_proximity')



In [26]:
data['ocean_proximity'].unique()

array(['INLAND', '<1H OCEAN', 'NEAR OCEAN', 'NEAR BAY', 'Rare'],
      dtype=object)

In [27]:
frequent_ls

Index(['<1H OCEAN', 'INLAND', 'NEAR BAY', 'NEAR OCEAN'], dtype='object', name='ocean_proximity')